In [1]:
import sudoku
reload(sudoku)

import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf

In [2]:
def matrix_to_one_hot(m):
    '''Make a 1-hot encoded version of an input sudoku matrix'''
    ret = np.zeros(9*9*9, dtype=int)
    for i, v in enumerate(m.flatten()):
        if v == 0:
            continue
        ret[i*9+v-1] = 1 
    return ret

def one_hot_to_matrix(oh):
    '''Make convert a one hot encoded sudoku matrix into a sudoku matrix'''
    ret = np.zeros(81, dtype=int)
    for i in range(81):
        val = -1
        for v in range(9):
            if oh[i*9+v] != 0:
                val = oh[i*9+v]
            if val != -1:
                ret[i] = v + 1
                break
    return ret.reshape(9,9)

def vec_to_one_hot(m):
    '''1 hot encode a vecotr of numbers between 0 and 9'''
    ret = np.zeros(9*9, dtype=int)
    for i, v in enumerate(m.flatten()):
        if v == 0:
            continue
        ret[i*9+v-1] = 1 
    return ret

In [3]:
import sudoku
reload(sudoku)
from collections import defaultdict

def generate_batch(n):
    '''Generate a set of sudoku puzzle steps with the next possible moves
    
    Returns:
        puz_x_features: the one hot encodeing of each 'box'
            by 'box' i mean a 1 hot encoding of each
                row (num 1-9)
                column (num 1-9)
                9x9 box (num 1-9)
        puz_y: the one hot encoded version of the next possible moves
        puz: the puzzle step (nice formatting)
    '''
    puzzles = []

    for _ in range(n):
        # create a sudoku puzzle, solve it, list each step
        #answer, steps to get to answer
        result, steps = sudoku.solve(sudoku.create_matrix())
        puzzle = steps
        
        # make sure the puzzle exists
        if puzzle is None:
            continue
        #sanity checks
        elif (not (sudoku.is_complete(puzzle[-1]) and 
                                 sudoku.is_correct(puzzle[-1]))):
            continue
        else:
            puzzles.append(puzzle)
    
    puz_x_features = []
    puz = []
    puz_y = []
    
    f1 = lambda x: x
    f2 = lambda x: np.flip(x, 1)
    f3 = lambda x: np.flip(x, 0)
    
    flips = [f1, f2, f3]
    
    # rot will rotate the matrix, this gives us cheap (CPU) training data
    for rot in range(0,4):
        for flip in flips:
            for puzzle in puzzles:
                previous = flip(np.rot90(puzzle[0], rot))
                for step in puzzle[1:-2]:
                    step = flip(np.rot90(step, rot))
                    if (step > 0).sum() == (previous > 0).sum():
                        continue

                    features = []

                    # loop though each column, row, box and create 1-hot encoded
                    # features for each
                    for i, v in enumerate(sudoku.enumerate_boxes(previous)):
                        features.extend(vec_to_one_hot(v))
                    for i, v in enumerate(sudoku.enumerate_columns(previous)):
                        features.extend(vec_to_one_hot(v))
                    for i, v in enumerate(sudoku.enumerate_rows(previous)):
                        features.extend(vec_to_one_hot(v))

                    puz_x_features.append(features)

                    # the delta between current and previous steps
                    # are the potential answers for this step
                    puz_y.append(matrix_to_one_hot(step - previous))

                    puz.append(previous)
                    previous = step
                
    return puz_x_features, puz_y, puz

In [4]:
puz_x_features, puz_y, puz = generate_batch(1)


input_size = len(puz_x_features[0])
output_size = puz_y[0].size
h1_size = int(output_size+(2./3.*input_size))


print input_size
print h1_size
print output_size

#9 (per box) * 9 (per value) * 9 (boxes) * 3 (box types)
print 9 * 9 * 9 * 3

2187
2187
729
2187


In [5]:
# hidden input rule of thumb tradeoff between num examples and hidden units
# https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
100000.*30/((9*9+729)*5)

740.7407407407408

In [6]:
tf.reset_default_graph()

x  = tf.placeholder(tf.float32, shape=[None, input_size])
y_ = tf.placeholder(tf.float32, shape=[None, output_size])


input_weights = []

# the weights for 1 hot encoded input values can be shared as 
# the logic for box, row, column should all be the same
with tf.variable_scope("w") as scope:
    for i in range(9*3):
        
        w = tf.get_variable("weights", [9*9, h1_size],
                initializer=tf.truncated_normal_initializer())
        if i == 0:
            scope.reuse_variables() 
        input_weights.append(w)
        
# concat all the shared weights
w1 = tf.concat(values=input_weights, axis=0)

# normal NN after this point
# perhaps ill add another hidden layer at some point
b1 = tf.Variable(tf.truncated_normal([h1_size], dtype=tf.float32))

w2 = tf.Variable(tf.truncated_normal([h1_size, output_size], dtype=tf.float32))
b2 = tf.Variable(tf.truncated_normal([output_size], dtype=tf.float32))

h1 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)

y = tf.matmul(h1, w2) + b2

# this allows us to guess for multiple cells at once
# softmax would allow us only to pick one
# the training data is set up to calculate all possible moves at each step
# so this is important
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y)

# arbitrary choice, need to learn more about this
train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cross_entropy)

# gpu was OOMing
session_conf = tf.ConfigProto(
    device_count={'CPU' : 1, 'GPU' : 0},
    allow_soft_placement=True,
    log_device_placement=True
)

sess = tf.InteractiveSession(config=session_conf)
tf.global_variables_initializer().run()

In [ ]:
# 100000 * batch(30 * 10 * 4)
for i in range(20000):
    if i % 300 == 0:
        print i
    puz_x_features, puz_y, puz = generate_batch(1)
    train_step.run(feed_dict={x: puz_x_features, y_: puz_y})

0


In [ ]:
puz_x_features, puz_y, puz = generate_batch(10)

res = y.eval(feed_dict={x: puz_x_features})

In [100]:
[v.name for v in tf.trainable_variables() ]

[u'w/weights:0', u'Variable:0', u'Variable_1:0', u'Variable_2:0']

In [101]:
var = [v for v in tf.trainable_variables() if v.name == "w/weights:0"][0].eval()

In [105]:
var.shape

(81, 2187)

In [106]:
w1

<tf.Tensor 'concat:0' shape=(2187, 2187) dtype=float32>

In [107]:
w1.eval()[81*0]

array([ 1.54567814,  1.04039156, -0.37916839, ...,  0.32666597,
        1.38302112, -0.11876588], dtype=float32)

In [108]:
w1.eval()[81*1]

array([ 1.54567814,  1.04039156, -0.37916839, ...,  0.32666597,
        1.38302112, -0.11876588], dtype=float32)

In [104]:
var[81*1]

IndexError: index 81 is out of bounds for axis 0 with size 81

In [49]:
var.eval().shape

(81, 2187)

In [62]:
input_size

2187

In [50]:
w1

<tf.Tensor 'concat:0' shape=(2187, 2187) dtype=float32>

In [38]:
var.eval().shape()

TypeError: 'tuple' object is not callable

In [36]:
var[0].eval()

array([ 0.38953662, -0.52640808, -1.22618234, ...,  0.70439446,
        0.99708056,  0.37677735], dtype=float32)

In [37]:
var[1].eval()

array([ 0.22519733,  0.50768501, -0.39286286, ..., -1.39292717,
       -0.33519465,  1.24322271], dtype=float32)

In [35]:
_.eval()

0.38953662

In [29]:
w1.

<tf.Tensor 'strided_slice_1:0' shape=() dtype=float32>

In [ ]:
check values of w1

In [4]:
input_size = 729
output_size = 729

In [7]:
import tensorflow as tf

tf.reset_default_graph()

x  = tf.placeholder(tf.float32, shape=[None, input_size])
y_ = tf.placeholder(tf.float32, shape=[None, output_size])

w1 = tf.Variable("w1", shape=[input_size, 100])
b1 = tf.Variable(tf.truncated_normal([100], dtype=tf.float32))

w3 = tf.Variable(tf.truncated_normal([100, output_size], dtype=tf.float32))
b3 = tf.Variable(tf.truncated_normal([output_size], dtype=tf.float32))

h1 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)
#h2 = tf.nn.sigmoid(tf.matmul(h1, w2) + b2)
y = tf.matmul(h1, w3) + b3

cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y)
train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cross_entropy)

session_conf = tf.ConfigProto(
    device_count={'CPU' : 1, 'GPU' : 0},
    allow_soft_placement=True,
    log_device_placement=True
)

sess = tf.InteractiveSession(config=session_conf)
tf.global_variables_initializer().run()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [8]:
for i in range(10000):
    x_train, y_train = generate_batch(100)
    train_step.run(feed_dict={x: x_train, y_: y_train})

RuntimeError: maximum recursion depth exceeded in cmp

In [9]:
i

0

In [ ]:
x_test, y_test = generate_batch(100)    
res = y.eval(feed_dict={x: x_train})

In [154]:
res.shape

(4, 729)

In [156]:
for am in res.argmax(axis=1):
    r = 

array([136,  19,  72, 120])

In [108]:

tn, fp, fn, tp = confusion_matrix(y_test, res).ravel()

In [109]:
(tn, fp, fn, tp)

(1532, 2, 91, 375)

In [115]:
reload(sudoku)
puzzles = []

for _ in range(10):
    result_steps = sudoku.solve(sudoku.create_matrix())
    puzzles.append(result_steps)

In [121]:
res = puzzles[0][1]

In [122]:
res[1] - res[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 9, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])